# Recursive Scraping with MultiOn

In this recipe, we'll demonstrate how to scrape URLs recursively using MultiOn's Retrieve API. We'll be using `retrieve` to scrape Hackernews posts and comments.

## Step 1: Set up the environment

First, let's install the required libraries and set up the MultiOn client.

In [ ]:
%pip install multion

In [3]:
from multion.client import MultiOn

client = MultiOn(
    api_key="YOUR_API_KEY   " # Get your API key from https://app.multion.ai/api-keys
)

## Step 2: Create the session

Next, we will create an agent session. Just to be safe, we first close all active sessions on our account.

In [4]:
for session_id in client.sessions.list().session_ids:
    client.sessions.close(session_id)

In [5]:
create_response = client.sessions.create(
    url="https://news.ycombinator.com/",
    local=False
)

session_id = create_response.session_id

## Step 3: Scrape Hackernews posts

We can start retrieving data with `retrieve`.

We will simply retrieve all the posts on the current page, along with information like the post's URL, score and URL for the comments section. Conveniently, `retrieve` has an option to scroll down to the bottom while retrieving (`scroll_to_bottom`), which we will use to speed up the process. We can also enable `render_js` to allow images (if any) to render.

In [6]:
scraped_posts = []

retrieve_response = client.retrieve(
    session_id=session_id,
    cmd="Get all the posts with the following fields: title, url, score, and comments url.",
    fields=["title", "url", "score", "comments_url"],
    scroll_to_bottom=True,
    render_js=True
)
scraped_posts.extend(retrieve_response.data)
print(f"Scraped page with {len(retrieve_response.data)} posts")

Scraped page with 30 posts


## Step 4: Recursively scrape comments for each post

We want to recursively get the comments for each post; we need to call `retrieve` on each individual post page. To speed up the process, we can use `retrieve` in parallel for each URL we previously scraped.

⚠️ We will be calling `retrieve` many times at once, beware of costs and rate limits!

In [ ]:
from concurrent.futures import ThreadPoolExecutor

def scrape_comments(post):
    post_comments_response = client.retrieve(
        url=post["comments_url"],
        cmd="Get all the comments, commenter names as array.",
        fields=["comment", "commenter"],
        max_items=5,
        scroll_to_bottom=True,
        render_js=True
    )
    post["comments"] = post_comments_response.data
    print(post['comments'])
    return post

scraped_posts = scraped_posts[:5]
with ThreadPoolExecutor(max_workers=5) as executor:
    scraped_posts = list(executor.map(scrape_comments, scraped_posts))

print(f"\n\nScraped detailed posts:\n{scraped_posts}")